In [13]:
!kubectl -n sqlserver get all

NAME                                   READY   STATUS    RESTARTS   AGE
pod/mssqlserver-2022-97999577d-s54s2   1/1     Running   0          3m17s

NAME                       TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/mssqlserver-2022   ClusterIP   10.101.35.21   <none>        1433/TCP   3m17s

NAME                               READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/mssqlserver-2022   1/1     1            1           3m17s

NAME                                         DESIRED   CURRENT   READY   AGE
replicaset.apps/mssqlserver-2022-97999577d   1         1         1       3m17s


In [18]:
!kubectl -n sqlserver get service/mssqlserver-2022 -o yaml | yq

apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: mssqlserver-2022
    meta.helm.sh/release-namespace: sqlserver
  creationTimestamp: "2024-01-29T17:18:56Z"
  labels:
    app: mssqlserver-2022
    app.kubernetes.io/managed-by: Helm
    chart: mssqlserver-2022-1.2.3
    heritage: Helm
    release: mssqlserver-2022
  name: mssqlserver-2022
  namespace: sqlserver
  resourceVersion: "30312"
  uid: 1edefb37-2672-4445-8104-0a29da240181
spec:
  clusterIP: 10.101.35.21
  clusterIPs:
    - 10.101.35.21
  internalTrafficPolicy: Cluster
  ipFamilies:
    - IPv4
  ipFamilyPolicy: SingleStack
  ports:
    - name: mssql
      port: 1433
      protocol: TCP
      targetPort: mssql
  selector:
    app: mssqlserver-2022
    release: mssqlserver-2022
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [15]:
!nmap mssqlserver-2022.sqlserver.svc 

Starting Nmap 7.80 ( https://nmap.org ) at 2024-01-29 15:15 -03
Nmap scan report for mssqlserver-2022.sqlserver.svc (10.101.35.21)
Host is up (0.00026s latency).
rDNS record for 10.101.35.21: mssqlserver-2022.sqlserver.svc.cluster2.xpt
Not shown: 999 closed ports
PORT     STATE SERVICE
1433/tcp open  ms-sql-s

Nmap done: 1 IP address (1 host up) scanned in 0.18 seconds


In [16]:
!kubectl -n sqlserver get secrets

NAME                                     TYPE                 DATA   AGE
mssqlserver-2022-secret                  Opaque               1      58m
sh.helm.release.v1.mssqlserver-2022.v1   helm.sh/release.v1   1      58m


In [4]:
!kubectl -n sqlserver get secrets mssqlserver-2022-secret -o yaml | yq

apiVersion: v1
data:
  sapassword: WkF5UHNNSzlSTTk5U2ZZZmF6Nlc=
kind: Secret
metadata:
  annotations:
    meta.helm.sh/release-name: mssqlserver-2022
    meta.helm.sh/release-namespace: sqlserver
  creationTimestamp: "2024-01-29T17:18:56Z"
  labels:
    app: mssqlserver-2022
    app.kubernetes.io/managed-by: Helm
    chart: mssqlserver-2022-1.2.3
    heritage: Helm
    release: mssqlserver-2022
  name: mssqlserver-2022-secret
  namespace: sqlserver
  resourceVersion: "30307"
  uid: 8b599acf-0a35-461c-95d7-ad3d7a9d2948
type: Opaque


# SA Password Retrieval

In [5]:
!kubectl -n sqlserver get secrets mssqlserver-2022-secret -o jsonpath="{.data.sapassword}" | base64 --decode

ZAyPsMK9RM99SfYfaz6W

In [18]:
!password="ZAyPsMK9RM99SfYfaz6W" && kubectl -n sqlserver run mssqlcli --image=mcr.microsoft.com/mssql-tools -ti --restart=Never --rm -- /opt/mssql-tools/bin/sqlcmd -S mssqlserver-2022.sqlserver.svc -U sa -P $password -Q "select @@VERSION;"

                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2022 (RTM-CU11) (KB5032679) - 16.0.4105.2 (X64) 
	Nov 14 2023 18:33:19 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 22.04.3 LTS) <X64>                                                                                                      

(1 rows affected)
pod "mssqlcli" deleted
